In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import gradio as gr


def load_model(model_path):
    model = models.densenet121(pretrained=False)
    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, 4)  # 4 classes
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.eval()
    return model

model = load_model("/kaggle/input/brain_tumor_mri_model/pytorch/default/1/brain_tumor_densenet_best.pth")
labels = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']


def predict(image):
    if image is None:
        return "Please upload an MRI scan image."

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
        _, pred = torch.max(outputs, 1)
    tumor_type = labels[pred.item()]
    return f"Predicted Category: {tumor_type}"


# 3. Gradio UI(gpt)

custom_css = """
.gradio-container {
    background-color: #f9fafc;
    font-family: 'Inter', sans-serif;
    color: #1e293b;
}
h1, h2, h3 {
    color: #0f172a !important;
}
.gr-button {
    background-color: #2563eb !important;
    color: white !important;
    border-radius: 8px !important;
    font-weight: 600 !important;
}
footer {
    visibility: hidden;
}
"""

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Brain MRI Scan"),
    outputs=gr.Textbox(label="AI Diagnostic Result"),
    title="AI-Powered Brain Tumor Classification System",
    description=(
        "This AI-driven diagnostic tool leverages a DenseNet121 deep learning model to classify "
        "MRI scans into one of four categories: Glioma, Meningioma, Pituitary, or No Tumor. "
        "Designed for clinical decision support and healthcare research environments."
    ),
    theme="gradio/soft",
    css=custom_css,
    allow_flagging="never",
    examples=None
)




if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://fcec7065ca9db92a7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
